In [1]:
import math
import random
import copy
import numpy as np

class PuzzleState:
    def __init__(self, puzzle_locations):
        self.puzzle_locations = puzzle_locations
        shape = puzzle_locations.shape
        self.y_size = shape[0]        
        self.x_size = shape[1]

    def get_location(self,label):
        x = -1
        y = -1
        for i in range(0, self.y_size):
            if (label in self.puzzle_locations[i, :]):
                y = i
        for i in range(0, self.x_size):
            if (label in self.puzzle_locations[:, i]):
                x = i
        return x, y

    def get_hash(self):
        count = int(0)
        sum = int(0)
        for i in self.puzzle_locations.flatten('C'):
            sum += i * math.pow((self.x_size*self.y_size),count)
            count += 1
        return int(sum)

    def __str__(self):
        return (self.puzzle_locations.__str__())

    def __eq__(self, other):
        if(other == None):
            return False
        return np.array_equal(self.puzzle_locations, other.puzzle_locations)


class PuzzleProblem:
    ACTIONS = {
        'Up': (0, -1),
        'Down': (0, 1),
        'Left': (-1, 0),
        'Right': (1, 0),
    }
    def __init__ (self,goal_state):
        self.goal_state = goal_state
    def test_goal(self,state):
        return state == self.goal_state
    def cost(self,node):
        if (node.parent == None):
            return 1
        return node.parent.cost+1
    #Manhatten
    def heuristic(self,state):
        flatten_state = self.goal_state.puzzle_locations.flatten('C')
        sum=0
        for label in flatten_state:
            x_goal,y_goal = self.goal_state.get_location(label)
            x_state,y_state = state.get_location(label)
            sum+=abs(x_goal-x_state)+(y_goal-y_state)
        return sum

    def test_fail(self,state): 
        return False
    def get_successors(self,state):
        result=[]
        for key in PuzzleProblem.ACTIONS.keys():
            action = key
            new_state =PuzzleProblem.transition(state,action)
            if(new_state != None):
                result.append((action,new_state))
        return result
    @staticmethod
    def transition(state: PuzzleState, action):
        new_state = copy.deepcopy(state)
        x_mov, y_mov = PuzzleProblem.ACTIONS[action]
        x_blank, y_blank = new_state.get_location(0)
        x_target = x_blank + x_mov
        y_target = y_blank + y_mov
        if(x_target < 0 or x_target > state.x_size - 1 or y_target < 0 or y_target > state.y_size-1):
            return None
        else:
            new_state.puzzle_locations[y_blank,
                                       x_blank] = new_state.puzzle_locations[y_target, x_target]
            new_state.puzzle_locations[y_target, x_target] = 0
        return new_state

    @staticmethod
    def shuffle(state, steps):
        count = 0
        shuffled_state = state
        while (count < steps):
            action = random.choice(list(PuzzleProblem.ACTIONS.keys()))
            new_state = PuzzleProblem.transition(shuffled_state, action)
            if(new_state != None):
                shuffled_state = new_state
                count += 1
        return shuffled_state
    
    
    
    
    
    import sys
from enum import Enum
from heapq import heappush, heappop

class Node:
    def __init__(self, parent, action, state, depth: int, score: float = float("inf"), cost: float = float("inf")):
        self.parent = parent
        self.action = action
        self.state = state
        self.depth = depth
        self.score = score
        self.cost = cost
    def get_neighbors(self):
        neighbors=[]
        return neighbors
    # This is needed for heap queue
    def __lt__(self, other):
        if(self.score == other.score):
            return self.action < other.action
        else:
            return self.score < other.score

class SearchMode(Enum):
    IDS = 0
    UCS = 1
    Greedy_BFS = 2
    A_Star = 3
    RBFS = 4

class SearchAgent:
    def __init__(self, start_state, problem, mode: SearchMode):
        self.start_state = start_state
        self.problem = problem
        self.explored = {}
        self.frontiers = []
        self.mode = mode
        self.max_queue_size =0
        self.max_queue_size = max(self.max_queue_size,len(self.frontiers))
        if(self.mode == SearchMode.IDS):
            self.cut_off_depth = 1
            self.cut_off_depth_reached = False

    def get_evaluate_value(self, node):
        # return a smaller number so it works like stack
        if(self.mode == SearchMode.IDS):
            if (len(self.frontiers) == 0):
                self.stack_count = 0
                return 0
            else:
                self.stack_count -= 1
                return self.stack_count
        # f(x) = g(x)
        elif (self.mode == SearchMode.UCS):
            return node.cost
        # f(x) = h(x)
        elif (self.mode == SearchMode.Greedy_BFS):
            return node.heuristic
        # f(x) = g(x) + h(x)
        elif (self.mode == SearchMode.A_Star):
            return node.cost+node.heuristic
        elif (self.mode == SearchMode.RBFS):
            return node.cost+node.heuristic
        return 0

    def process_node(self, node: Node):
        state = node.state
        hash = state.get_hash()
        if (self.problem.test_goal(state)):
            return node
        if (hash in self.explored.keys()):
            return None
        if (self.problem.test_fail(node)):
            return None
        self.explored[hash] = node

        # For Iterative-Deepening Search, stop adding frontier if the depth reach cut off depth
        if(self.mode == SearchMode.IDS and node.depth >= self.cut_off_depth):
            # create a new search agent with larger cut off depth
            self.cut_off_depth_reached=True
            return None
        new_nodes =[]
        for successor in self.problem.get_successors(node.state):
            action,state = successor
            new_node = Node(node,action,state,node.depth+1)
            new_node.cost = self.problem.cost(new_node)
            new_node.heuristic = self.problem.heuristic(new_node.state)
            new_node.score = self.get_evaluate_value(new_node)
            new_nodes.append(new_node)
        for new_node in new_nodes:
            self.add_frontier(new_node)
        return None

    def add_frontier(self, node):
        heappush(self.frontiers, node)
        self.max_queue_size = max(len(self.frontiers), self.max_queue_size)

    def get_next_frontier(self) -> Node:
        return heappop(self.frontiers)
    def process_rbfs(self,node,f_limit):
        max_queue_size=0
        if (self.problem.test_goal(node.state)):
            return node,node.score,max_queue_size
        new_nodes=[]
        for successor in self.problem.get_successors(node.state):
            action,state = successor
            new_node = Node(node,action,state,node.depth+1)
            new_node.cost = self.problem.cost(new_node)
            new_node.heuristic = self.problem.heuristic(new_node.state)
            new_node.score = self.get_evaluate_value(new_node)
            new_node.score = max(new_node.score,node.score)
            new_nodes.append(new_node)
        if (len(new_nodes)==0):
            return None,float('inf'),0
        while (1):
            new_nodes.sort(key=lambda n:n.score)
            best = new_nodes[0]
            if(best.score > f_limit):
                return None,best.score,0
            if (len(new_nodes)>=2):
                alternative = new_nodes[1].score
            else:    
                alternative = float('inf')
            result,best.score,new_queue_size = self.process_rbfs(best,min(f_limit,alternative))
            max_queue_size=new_queue_size+len(new_nodes)
            if(result!=None):
                return result,result.score,max_queue_size
        return None,float('inf'),max_queue_size

    def search(self):
        root_node = Node(None, None, self.start_state,0,cost=0)
        root_node.heuristic = self.problem.heuristic(root_node.state)
        root_node.score = self.get_evaluate_value(root_node)
        if(self.mode == SearchMode.RBFS):
            solution_node,_,self.max_queue_size = self.process_rbfs(root_node,float('inf'))
        else:
            self.add_frontier(root_node)
            while (len(self.frontiers) > 0):
                frontier_node = self.get_next_frontier()
                solution_node = self.process_node(frontier_node)
                if(solution_node != None):
                    break
        if (solution_node != None):
            print('explored: ', len(self.explored))
            print(
                f"Mode: {self.mode} Solution found!, Max queue size: {self.max_queue_size} Max depth: {solution_node.depth}")
            actions =[]
            node = solution_node
            while (node!=None):
                actions.append(node.action)
                node = node.parent
            actions.reverse()
            #Skip root action                           
            actions = actions[1:]
            print (actions)
            return solution_node
        if(self.mode == SearchMode.IDS and self.cut_off_depth_reached):
        #create a new search agent with larger cut off depth
            new_agent = SearchAgent(self.start_state,self.problem,self.mode)
            new_agent.max_queue_size = self.max_queue_size
            new_agent.cut_off_depth = self.cut_off_depth+1
            return new_agent.search()
        print(f"{self.mode} Failed!, Max queue size: {self.max_queue_size}")
        return None
    
    
    
    import numpy as np



goal_locations = np.array([[1,2,3],[8,0,4],[7,6,5]])
start_state = np.array([[8,3,5],[4,1,6],[2,7,0]])
state2 = PuzzleState(start_state)
goal_state = PuzzleState(goal_locations)

start_state = PuzzleProblem.shuffle(state2,0)
#start_state =([1,2,3],[4,6,5],[7,8,0])
problem = PuzzleProblem(goal_state)

print('Goal State')
print(goal_state)
print('Start State')
print(start_state)

agent = SearchAgent(start_state, problem, SearchMode.UCS)
solution_node = agent.search()

# agent = SearchAgent(start_state, problem, SearchMode.Greedy_BFS)
# solution_node = agent.search()

# agent = SearchAgent(start_state, problem, SearchMode.A_Star)
# solution_node = agent.search()

# agent = SearchAgent(start_state, problem, SearchMode.IDS)
# solution_node = agent.search()

# agent = SearchAgent(start_state, problem, SearchMode.RBFS)
# solution_node = agent.search()

Goal State
[[1 2 3]
 [8 0 4]
 [7 6 5]]
Start State
[[8 3 5]
 [4 1 6]
 [2 7 0]]
Mode: SearchMode.UCS Solution found!, Max queue size: 3224 Max depth: 14
['Up', 'Up', 'Left', 'Down', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left', 'Up', 'Right', 'Down']
